# Información previa



## datos
faltan en pollo 1010130 y 1010387

Link de descarga de BW

[08_06_21_Fact_CL_Mater_Canales_10_06_21](https://gestion.agrosuper.cl/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=4V7XT6V546W73QKC7BVTLKF4Y)



## Diseño y esquema codigo

## Apuntes Cod 

```{python}
Data['Peso neto KG']=Data['Peso neto'].str.split('KG', expand=True)[[0]]
Data.pop('Peso neto')
Data['Cantidad Unidades CJ']=Data['Cantidad Unidades'].str.split('CJ', expand=True)[[0]]
Data.pop('Cantidad Unidades')
Data['Precio Unitario CLP / KG']=Data['Precio Unitario'].str.split('CLP / KG', expand=True)[[0]]



    if(x[len(x)-3:]=='USD'):
        x=x.split(' USD')[1]
    elif(x[len(x)-3:]=='CLP'):
        x=x.split(' CLP')[1]


```

## Este es un extracto del paper:


 

	
Ventas pasadas con retraso  k, es decir, ventas diarias para el SKU considerado de los últimos k días (tw k : ventana de tiempo de tamaño k ).

	
* Variables binarias para caracterizar días especiales, como:
    * Pago , que caracteriza los días al final de cada mes cuando las personas reciben el pago mensual.
    * Pago intermedio , que caracteriza los días al final de cada quincena en que las personas reciben dos pagos semanales.
    * Antes de las vacaciones , caracterizando los días antes de las vacaciones.
    * Vacaciones , que caracterizan las vacaciones.
    * Independencia , que caracteriza los días de la independencia de Chile (18 y 19 de septiembre).
    * Santa , caracterizando los días de la semana antes del este.
    * Vacaciones , caracterizando los días que pertenecen al período de vacaciones de verano (enero y febrero).
    * Verano , que caracteriza los días de verano (1 de octubre-31 de marzo).	
    * Año nuevo (1 de enero), que caracteriza el único día de cierre de supermercados en Chile.
	
	
* Variables de precio:
    * Precio A  =  precio original del SKU considerado en Economax.
    * Precio B  =  (Precio A) / (Precio máximo en micro-mercado).
    * Precio C  =  (Precio A) / (Precio mínimo en micro-mercado).

		
Observación : Las variables “Precio máximo en micro-mercado” y “Precio mínimo en micro-mercado” son el precio máximo (mínimo) para un determinado SKU en el micro-mercado relevante y son proporcionados por una empresa de investigación de mercado.
		
	

	
 

# Package Install & Open Data


## Import

### import general 

In [1]:
# =============================================================================
# Importar paquetes y abrir Data
# =============================================================================

#gestion DF
import pandas as pd

#Funciones mat y matricicales
import numpy as np

#ploteo simple
import matplotlib.pyplot as plt

# Graficas bonitas
from plotnine import *

#para ajustar las fechas
import datetime

#para calcular el skew de un array() o list()
from scipy.stats import skew 

#mode function
import statistics 

from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [2]:
from time import time

### import and function 

In [3]:

import warnings
import pandas as pd
import matplotlib.pyplot as plt
# prophet model 
from fbprophet import Prophet
# prophet preformance
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

# don't do this 
warnings.filterwarnings('ignore')
# "high resolution"
%config InlineBackend.figure_format = 'retina'

Importing plotly failed. Interactive plots will not work.


##  Data Facturación

Link de descarga de BW

[08_06_21_Fact_CL_Mater_Canales_10_06_21](https://gestion.agrosuper.cl/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=4V7XT6V546W73QKC7BVTLKF4Y)

### Selección de la zona de estudio 

In [1]:
#Nombre del producto
Fecha='08_06_21'

# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name='Fact_CL_Mater_Canales_'+Fecha
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=";",dtype=types_dict)
Data.head(1)

NameError: name 'pd' is not defined

### Ajuste facturaciones

In [5]:
# =============================================================================
# Ajustar la data
# =============================================================================
def Ajuste_data(data):
    df=data.copy()
    # Funciones para eliminar los putos y cambiar las , por .
    def categ_uniq(x):
        return x[len(x)-3:]

    df['Unidad Venta']=df['Venta Neta'].apply(categ_uniq)
    df['Unidad Precio']=df['Precio Promedio'].apply(categ_uniq)
    
    def CLP_Dolar(x):
        x=str(x)
        x=x.replace('.', '')
        x=x.replace(',', '.')
        if(x[len(x)-3:]=='USD'):
            x=x.split(' USD')[0]
            x=float(x)
            #x=x*720
        elif(x[len(x)-3:]=='CLP'):
            x=x.split(' CLP')[0]
            x=float(x)
        return x
    
    def change_num_point(x):
        x=str(x)
        return x.replace('.', '')
    def change_num_coma(x):
        x=str(x)
        return x.replace(',', '.')
    def delete_asterisco(x):
        x=str(x)
        return x.replace('*','0' )
    
    #proceso iterativo, 
    for i in ['Kilos Venta KG','Venta Neta','Precio Promedio']:
    # se elimina el punto definido para los miles
        if(i!='Kilos Venta KG'):
            df[i]=df[i].apply(CLP_Dolar)
        else:
            df[i]=df[i].apply(change_num_point)
            # se reemplaza la coma por un punto, para definir los decimales correctamente
            df[i]=df[i].apply(change_num_coma)
        # se elimina el valor * 
        df[i]=df[i].apply(delete_asterisco)
        df[i]=df[i].astype('float64')
        # Se pasa de ser un str a un int
        print(i)
        #df[i]=df[i].astype('int64')

    #Separacion fecha
    df['Semana']=df['Año natural/Semana'].str.split('.', expand=True)[[0]]
    df['Año']=df['Año natural/Semana'].str.split('.', expand=True)[[1]]

    df['Semana']=df['Semana'].astype('int64')
    df['Año']=df['Año'].astype('int64')
    
    #
    df["Día natural"] = df["Día natural"].apply(lambda x: 
                                                datetime.datetime.strptime(x,
                                                                           '%d.%m.%Y').date())
    df['Año']=df['Año'].astype('int64')
    return df

In [6]:
data1=Ajuste_data(Data)
data1.head(3)
#data=<data[data['Año']>2018]

Kilos Venta KG
Venta Neta
Precio Promedio


,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019


##  Data Facturación Concatena a la anterior
Link de descarga de BW
[08_06_21_Fact_CL_Mater_Canales_10_06_21](https://gestion.agrosuper.cl/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=4V7XT6V546W73QKC7BVTLKF4Y)

Contiene las facturaciones siguientes de todo chile para el listado de productos y para los canales de venta:
Foodservices; Canal Tradicional; Industriales; GRandes Clientes

### Selección de la zona de estudio 

In [7]:
#Nombre del producto
Fecha='08_06_21'
Fecha_fin='12_06_21'
# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name=Fecha+'_Fact_CL_Mater_Canales_'+Fecha_fin
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data2 = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=";",dtype=types_dict)
Data2.head(1)

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,6,23.2021,08.06.2021,MA,273,697.541 CLP,2.555 CLP


In [8]:
data2=Ajuste_data(Data2)

Kilos Venta KG
Venta Neta
Precio Promedio


## Unificar 

### Primera y ultima fecha 

In [9]:
data2['Día natural'].max()

datetime.date(2021, 6, 11)

In [10]:
data2['Día natural'].min()

datetime.date(2021, 6, 8)

In [11]:
data1['Día natural'].max()

datetime.date(2021, 6, 7)

In [12]:
data1['Día natural'].min()

datetime.date(2019, 1, 2)

### Concatenar 

In [13]:
data=pd.concat([data1, data2])
data.describe(include='all')

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
count,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430.000000,4.494300e+05,449430.000000,449430,449430,449430.000000,449430.000000
unique,2,13,4,6,6,34,34,12,128,788,7,NaN,NaN,NaN,3,3,NaN,NaN
top,Pollo,1010078,Tradicional,Z003,Centro Sur,Concepcion,T104,3,49.2019,2019-04-18,MI,NaN,NaN,NaN,CLP,CLP,NaN,NaN
freq,238796,52145,210463,124471,124471,22980,22980,49744,4061,731,77977,NaN,NaN,NaN,449015,449015,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.307383,9.601389e+05,1849.338858,NaN,NaN,23.717983,2019.762070
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1245.086148,1.802542e+06,724.569747,NaN,NaN,15.067890,0.732747
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,NaN,NaN,1.000000,2019.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.000000,1.205830e+05,1307.000000,NaN,NaN,11.000000,2019.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.000000,3.469035e+05,1686.000000,NaN,NaN,22.000000,2020.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,555.000000,9.725628e+05,2278.000000,NaN,NaN,36.000000,2020.000000


In [14]:
data['Día natural'].min()

datetime.date(2019, 1, 2)

In [15]:
data['Día natural'].max()

datetime.date(2021, 6, 11)

# Predicción  Prophet


In [16]:
y_col='Kilos Venta KG'
y_value='Volumen de ventas en KG'
y_title= 'Demanda'+'del canal tradicion'

## Descripción 

In [17]:
data.head()

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019
3,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-07,LU,128.0,258591.0,2017.0,CLP,CLP,2,2019
4,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-08,MA,95.0,210790.0,2223.0,CLP,CLP,2,2019


In [17]:
data.columns

Index(['Sector', 'Material', 'Tipo de cliente', 'Zona de ventas',
       'Zona de ventas.1', 'Oficina de Ventas Unifica',
       'Oficina de Ventas Unifica.1', 'Mes', 'Año natural/Semana',
       'Día natural', 'Día de la semana', 'Kilos Venta KG', 'Venta Neta',
       'Precio Promedio', 'Unidad Venta', 'Unidad Precio', 'Semana', 'Año'],
      dtype='object')

In [18]:
print(data['Tipo de cliente'].unique())
print(data['Zona de ventas.1'].unique())
print(data['Oficina de Ventas Unifica'].unique())
print(data['Material'].unique())

['Tradicional' 'Industriales' 'Grandes Clientes' 'Foodservice']
['Santiago' 'Centro Norte' 'Centro Sur' 'Sur' 'Norte' 'Venta Directa']
['Huechuraba' 'Lo Espejo' 'Quilin' 'Sucursal Miraflores' 'Coquimbo'
 'Hijuelas' 'Ovalle' 'San Antonio' 'San Felipe' 'Viña del Mar' 'Chillan'
 'Concepcion' 'Curico' 'Los Angeles' 'Rancagua' 'Talca' 'Castro'
 'Coyhaique' 'Osorno' 'Puerto Montt' 'Punta Arenas' 'Temuco' 'Valdivia'
 'Antofagasta' 'Arica' 'Calama' 'Copiapo' 'Iquique' 'Vallenar'
 'VD FoodService' 'VD Industrial' 'VD Grandes Clientes'
 'Sucursal Dieciocho' 'Vendedores San Pablo']
['1020253' '1020042' '1020027' '1020013' '1020010' '1020003' '1020001'
 '1010695' '1010267' '1010078' '1010077' '1010048' '1010002']


## función 

In [19]:
def Df_ubic(data_X,col,Str):
    df=data_X.copy()
    df=df[df[col]==Str]
    return df

In [20]:
def Predict_sem(D_Train_x,periodos=5):
    
    fecha_i=datetime.date(2019, 1, 1)
    listado_fechas=[fecha_i]
    while(fecha_i<datetime.date(2021, 6, 12)):
        fecha_i=fecha_i+datetime.timedelta(days=1)
        listado_fechas.append(fecha_i)
    df_fechas=pd.DataFrame({'Día natural':listado_fechas})
    
    D_Train=pd.DataFrame.merge(D_Train_x,df_fechas,how='right').fillna(0)
    
    
    D_Train.columns = ['ds','y']
    
    # convert dates from string to datetime
    D_Train.ds = pd.to_datetime(D_Train.ds)
    #print(D_Train.dtypes)
    
    # set prophet model 
    prophet = Prophet(changepoint_prior_scale=0.15, yearly_seasonality=True)
    
    # fit $wmt data to model
    prophet.fit(D_Train)
    
    
    
    # build future dataframe for 5 years
    build_forecast = prophet.make_future_dataframe(periods= periodos , freq='D')
    #prophet.make_future_dataframe(periods= periodos, freq='D')
    
    # forecast future df w/ model
    forecast = prophet.predict(build_forecast)
    
    return(forecast)

### Listado de fechas 

In [21]:
df4=Df_ubic(data,'Material','1020003'
            ).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]
df4.head

<bound method NDFrame.head of     Día natural  Kilos Venta KG
0    2019-01-02           331.0
1    2019-01-03          1871.0
2    2019-01-04          3771.0
3    2019-01-05          2787.0
4    2019-01-07          1352.0
..          ...             ...
731  2021-06-07          2459.0
732  2021-06-08          6725.0
733  2021-06-09          5278.0
734  2021-06-10          7215.0
735  2021-06-11          7355.0

[736 rows x 2 columns]>

In [22]:
df4.iloc[0,0]

datetime.date(2019, 1, 2)

In [27]:
datetime.date(2019, 1, 2)-datetime.date(2019, 1, 1)

datetime.timedelta(days=1)

In [41]:
fecha_i=datetime.date(2019, 1, 1)
listado_fechas=[fecha_i]
while(fecha_i<datetime.date(2021, 6, 12)):
    fecha_i=fecha_i+datetime.timedelta(days=1)
    listado_fechas.append(fecha_i)
df_fechas=pd.DataFrame({'Día natural':listado_fechas})

In [42]:
listado_fechas

[datetime.date(2019, 1, 1),
 datetime.date(2019, 1, 2),
 datetime.date(2019, 1, 3),
 datetime.date(2019, 1, 4),
 datetime.date(2019, 1, 5),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 7),
 datetime.date(2019, 1, 8),
 datetime.date(2019, 1, 9),
 datetime.date(2019, 1, 10),
 datetime.date(2019, 1, 11),
 datetime.date(2019, 1, 12),
 datetime.date(2019, 1, 13),
 datetime.date(2019, 1, 14),
 datetime.date(2019, 1, 15),
 datetime.date(2019, 1, 16),
 datetime.date(2019, 1, 17),
 datetime.date(2019, 1, 18),
 datetime.date(2019, 1, 19),
 datetime.date(2019, 1, 20),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 1, 22),
 datetime.date(2019, 1, 23),
 datetime.date(2019, 1, 24),
 datetime.date(2019, 1, 25),
 datetime.date(2019, 1, 26),
 datetime.date(2019, 1, 27),
 datetime.date(2019, 1, 28),
 datetime.date(2019, 1, 29),
 datetime.date(2019, 1, 30),
 datetime.date(2019, 1, 31),
 datetime.date(2019, 2, 1),
 datetime.date(2019, 2, 2),
 datetime.date(2019, 2, 3),
 datetime.date(2019, 2, 4)

In [43]:
df_fechas=pd.DataFrame({'Día natural':listado_fechas})
df_fechas

,Día natural
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05
...,...
889,2021-06-08
890,2021-06-09
891,2021-06-10
892,2021-06-11


In [45]:
pd.DataFrame.merge(df4,df_fechas,how='right').fillna(0)

,Día natural,Kilos Venta KG
0,2019-01-01,0.0
1,2019-01-02,331.0
2,2019-01-03,1871.0
3,2019-01-04,3771.0
4,2019-01-05,2787.0
...,...,...
889,2021-06-08,6725.0
890,2021-06-09,5278.0
891,2021-06-10,7215.0
892,2021-06-11,7355.0


## Predicción de todo chile

In [22]:
#inicializa con un Data Frame vacio
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat',
                            'Material'])

df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

#Crear un listado de los materiales disponibles en la data
Listado_de_Materiales=data['Material'].unique()

for m in Listado_de_Materiales:
    #m=Listado_de_Materiales[0]
    dfm=Df_ubic(data,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]
    
    #Realizar predicción
    dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(7)
    dfmPred.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)
    dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
    
    #Sector del producto(Tipo de carne)
    Sec=Df_ubic(data,'Material',m).Sector.unique()[0]
    dfmPred['Sector']=[Sec for i in range(dfmPred.shape[0])]
    
    dfmPred=dfmPred.reset_index()
    dfmPred.pop('index')
    df2=pd.concat([df2, dfmPred])

    
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Sector
0,2021-06-12,8968.107391,18413.527409,13680.623828,1020253,Cerdo
1,2021-06-13,2158.138173,10842.142436,6663.595223,1020253,Cerdo
2,2021-06-14,6243.877989,15022.123846,10707.495410,1020253,Cerdo
3,2021-06-15,8137.727950,17507.610926,12744.569866,1020253,Cerdo
4,2021-06-16,9629.103873,18862.950516,14287.205309,1020253,Cerdo


### CSV SAVE

In [23]:

min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-12
2021-06-18


In [24]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Todo Chile del'+min_time+' al '+max_time+'.csv', index=False)

### Analisis Exploratorio

In [25]:
df2.groupby(['Material','Sector']).sum()

,,Predicción Inferior,Predicción Superior,Predicción central
Material,Sector,,,
1010002,Pollo,68586.041678,2.243420e+05,145637.183903
1010048,Pollo,27760.452515,3.128215e+05,169019.134438
1010077,Pollo,9336.688060,5.213126e+04,30798.927772
1010078,Pollo,577479.698548,1.052460e+06,819076.146304
1010267,Pollo,134399.679466,3.405101e+05,237439.575885
1010695,Pollo,8442.894416,1.367922e+05,71965.008433
1020001,Cerdo,47826.876464,1.170952e+05,82568.853232
1020003,Cerdo,1772.521362,7.351906e+04,37819.226240
1020010,Cerdo,138222.578529,2.526346e+05,195783.956847


## Predicción POR CANAL EN todo chile

In [26]:
data.columns

Index(['Sector', 'Material', 'Tipo de cliente', 'Zona de ventas',
       'Zona de ventas.1', 'Oficina de Ventas Unifica',
       'Oficina de Ventas Unifica.1', 'Mes', 'Año natural/Semana',
       'Día natural', 'Día de la semana', 'Kilos Venta KG', 'Venta Neta',
       'Precio Promedio', 'Unidad Venta', 'Unidad Precio', 'Semana', 'Año'],
      dtype='object')

In [27]:
#inicializa con un Data Frame vacio
df3 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat',
                            'Material'])

Listado_de_Canales=data['Tipo de cliente'].unique()

for c in Listado_de_Canales:
    #m=Listado_de_Materiales[0]
    dfc=Df_ubic(data,'Tipo de cliente',c)

    #Crear un listado de los materiales disponibles en la data
    Listado_de_Materiales=dfc[dfc.Año==2021].Material.unique()     


    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        dfm=Df_ubic(dfc,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

        #Realizar predicción
        dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(7)

        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]

        #Sector del producto(Tipo de carne)
        Sec=Df_ubic(data,'Material',m).Sector.unique()[0]
        dfmPred['Sector']=[Sec for i in range(dfmPred.shape[0])]
        
        #Canal de ventas
        dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]
        
        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df3=pd.concat([df3, dfmPred])

    
df3.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,ds,yhat_lower,yhat_upper,yhat,Material,Sector,Tipo de cliente
0,2021-06-12,6387.063999,13585.556490,9947.250821,1020253,Cerdo,Tradicional
1,2021-06-13,-94.091794,7563.537323,3799.199531,1020253,Cerdo,Tradicional
2,2021-06-14,3432.973315,10709.050944,7040.434877,1020253,Cerdo,Tradicional
3,2021-06-15,4889.102061,11760.252280,8256.586063,1020253,Cerdo,Tradicional
4,2021-06-16,6163.229082,13288.319165,9845.337275,1020253,Cerdo,Tradicional


In [28]:
df3.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [29]:
df3.tail()

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Sector,Tipo de cliente
2,2021-06-14,-814.097124,2154.812658,670.675854,1010002,Pollo,Foodservice
3,2021-06-15,-253.487008,2619.781814,1273.466913,1010002,Pollo,Foodservice
4,2021-06-16,83.445143,2994.927289,1570.806590,1010002,Pollo,Foodservice
5,2021-06-17,206.754764,3059.690572,1664.442530,1010002,Pollo,Foodservice
6,2021-06-18,156.278989,3019.345234,1609.255150,1010002,Pollo,Foodservice


### CSV SAVE

In [30]:
min_time=df3['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df3['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-10
2021-06-18


In [31]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df3.to_csv('Predicción Todo Chile del '+min_time+' al '+max_time+' Diferenciando por canal.csv', index=False)

### Exploracion 

In [32]:
D_sem=df3.groupby(['Sector','Material','Tipo de cliente']).sum().reset_index()
D_sem

,Sector,Material,Tipo de cliente,Predicción Inferior,Predicción Superior,Predicción central
0,Cerdo,1020001,Foodservice,-731.960533,1981.151618,620.056258
1,Cerdo,1020001,Grandes Clientes,2845.796715,18292.457537,10505.959754
2,Cerdo,1020001,Industriales,-2738.533189,3751.204081,517.133608
3,Cerdo,1020001,Tradicional,42843.066105,99327.087411,70960.690166
4,Cerdo,1020003,Foodservice,924.330928,6035.413995,3480.290090
5,Cerdo,1020003,Grandes Clientes,-7908.796274,10357.068202,1410.917086
6,Cerdo,1020003,Industriales,-2069.811977,3955.477692,935.025980
7,Cerdo,1020003,Tradicional,4200.775505,59012.410078,31418.655539
8,Cerdo,1020010,Foodservice,5427.250754,29335.543443,17438.700606
9,Cerdo,1020010,Grandes Clientes,11441.780011,44132.313442,27662.844443


In [33]:
D_sem[D_sem.Material=='1010130']

,Sector,Material,Tipo de cliente,Predicción Inferior,Predicción Superior,Predicción central


In [34]:
df3.groupby(['Sector','Material','Tipo de cliente']).sum()

Predicción Inferior  Predicción Superior  \
Sector Material Tipo de cliente                                              
Cerdo  1020001  Foodservice               -731.960533          1981.151618   
                Grandes Clientes          2845.796715         18292.457537   
                Industriales             -2738.533189          3751.204081   
                Tradicional              42843.066105         99327.087411   
       1020003  Foodservice                924.330928          6035.413995   
                Grandes Clientes         -7908.796274         10357.068202   
                Industriales             -2069.811977          3955.477692   
                Tradicional               4200.775505         59012.410078   
       1020010  Foodservice               5427.250754         29335.543443   
                Grandes Clientes         11441.780011         44132.313442   
                Industriales             -6246.376838         10277.439560   
                Tradicional             103848.010989        183521.127156   
       1020013  Foodservice               -225.366832          7309.720924   
                Grandes Clientes         13166.800075         91106.411050   
                Industriales             -9367.630079         17180.693468   
                Tradicional             142454.033567        297909.099461   
       1020027  Foodservice               -518.312180          4228.797859   
                Grandes Clientes          9594.030893         30964.481091   
                Industriales             -2726.939040          7961.863723   
                Tradicional              56130.351765        125185.385273   
       1020042  Foodservice               -490.459349          1213.182158   
                Grandes Clientes         -1787.401748         10104.403676   
                Industriales              3445.257017         12916.966309   
                Tradicional               1659.332970         25150.479238   
       1020253  Foodservice              12398.920214         23328.005195   
                Grandes Clientes          -260.337302         13993.929056   
                Industriales              -339.119666          6636.300303   
                Tradicional              35719.133699         86551.403997   
Pollo  1010002  Foodservice              -2527.139750         17820.011236   
                Grandes Clientes           235.943088         43058.242167   
                Industriales             -1011.316079          2705.575401   
                Tradicional              61159.822650        181621.638371   
       1010048  Foodservice              15911.398491         46592.753680   
                Grandes Clientes        -46330.151512         53282.137598   
                Industriales              -485.470107          4363.061660   
                Tradicional              30394.719050        233131.963062   
       1010077  Foodservice               -112.542732          3974.301436   
                Grandes Clientes         -4341.164087          3012.496373   
                Industriales              -946.119843           765.937673   
                Tradicional               9701.213697         47670.883867   
       1010078  Foodservice              42634.069470         72676.637554   
                Grandes Clientes          4693.191791        135698.296249   
                Industriales              1030.719562         17260.938874   
                Tradicional             501296.864785        872217.179219   
       1010267  Foodservice              96249.060791        189881.812568   
                Grandes Clientes        -25977.036601         25754.959190   
                Industriales            -25192.320494         21186.516176   
                Tradicional              29826.847124        151128.391581   
       1010695  Foodservice               4345.676826         15688.158634   
                Grandes Clientes        -19360.798554   

In [35]:
df3.groupby(['Sector','Material','Tipo de cliente']).sum()

Predicción Inferior  Predicción Superior  \
Sector Material Tipo de cliente                                              
Cerdo  1020001  Foodservice               -731.960533          1981.151618   
                Grandes Clientes          2845.796715         18292.457537   
                Industriales             -2738.533189          3751.204081   
                Tradicional              42843.066105         99327.087411   
       1020003  Foodservice                924.330928          6035.413995   
                Grandes Clientes         -7908.796274         10357.068202   
                Industriales             -2069.811977          3955.477692   
                Tradicional               4200.775505         59012.410078   
       1020010  Foodservice               5427.250754         29335.543443   
                Grandes Clientes         11441.780011         44132.313442   
                Industriales             -6246.376838         10277.439560   
                Tradicional             103848.010989        183521.127156   
       1020013  Foodservice               -225.366832          7309.720924   
                Grandes Clientes         13166.800075         91106.411050   
                Industriales             -9367.630079         17180.693468   
                Tradicional             142454.033567        297909.099461   
       1020027  Foodservice               -518.312180          4228.797859   
                Grandes Clientes          9594.030893         30964.481091   
                Industriales             -2726.939040          7961.863723   
                Tradicional              56130.351765        125185.385273   
       1020042  Foodservice               -490.459349          1213.182158   
                Grandes Clientes         -1787.401748         10104.403676   
                Industriales              3445.257017         12916.966309   
                Tradicional               1659.332970         25150.479238   
       1020253  Foodservice              12398.920214         23328.005195   
                Grandes Clientes          -260.337302         13993.929056   
                Industriales              -339.119666          6636.300303   
                Tradicional              35719.133699         86551.403997   
Pollo  1010002  Foodservice              -2527.139750         17820.011236   
                Grandes Clientes           235.943088         43058.242167   
                Industriales             -1011.316079          2705.575401   
                Tradicional              61159.822650        181621.638371   
       1010048  Foodservice              15911.398491         46592.753680   
                Grandes Clientes        -46330.151512         53282.137598   
                Industriales              -485.470107          4363.061660   
                Tradicional              30394.719050        233131.963062   
       1010077  Foodservice               -112.542732          3974.301436   
                Grandes Clientes         -4341.164087          3012.496373   
                Industriales              -946.119843           765.937673   
                Tradicional               9701.213697         47670.883867   
       1010078  Foodservice              42634.069470         72676.637554   
                Grandes Clientes          4693.191791        135698.296249   
                Industriales              1030.719562         17260.938874   
                Tradicional             501296.864785        872217.179219   
       1010267  Foodservice              96249.060791        189881.812568   
                Grandes Clientes        -25977.036601         25754.959190   
                Industriales            -25192.320494         21186.516176   
                Tradicional              29826.847124        151128.391581   
       1010695  Foodservice               4345.676826         15688.158634   
                Grandes Clientes        -19360.798554   

## Predicción selecionada 

### Sucursal 

In [23]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])


Listado_de_zonas=data['Zona de ventas.1'].unique()

#for z in Listado_de_zonas:
Z='Norte'

dfz=data#Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz['Oficina de Ventas Unifica'].unique()

#for y in Listado_de_Sucursales:

y='Arica'
dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


Listado_de_Materiales=dfy['Material'].unique()

#for m in Listado_de_Materiales:
m=Listado_de_Materiales[0]
dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
dfmPred=dfmPred.reset_index()
dfmPred.pop('index')
df2=pd.concat([df2, dfmPred])
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-14,-39.126508,111.259811,33.700802,1020253,Arica,Norte
1,2021-06-15,-39.179534,105.034340,33.941449,1020253,Arica,Norte
2,2021-06-16,-26.405262,130.290545,53.885961,1020253,Arica,Norte
3,2021-06-17,-19.319263,125.727325,51.228343,1020253,Arica,Norte
4,2021-06-18,-19.550630,135.658925,55.227983,1020253,Arica,Norte


In [29]:
dfm[dfm['Día natural']>datetime.date(2021, 1, 2)
    ].shape[0]>20

True

### Zona 

In [37]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])


Listado_de_zonas=data['Zona de ventas.1'].unique()

#for z in Listado_de_zonas:
Z='Santiago'

dfz=data#Df_ubic(data,'Zona de ventas.1',Z)


Listado_de_Materiales=dfz['Material'].unique()

#for m in Listado_de_Materiales:
m=Listado_de_Materiales[0]
dfm=Df_ubic(dfz,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
#dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
dfmPred=dfmPred.reset_index()
dfmPred.pop('index')
df2=pd.concat([df2, dfmPred])
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-13,2255.909436,11048.439429,6663.595223,1020253,NaN,Santiago
1,2021-06-14,6155.265265,14977.040561,10707.495410,1020253,NaN,Santiago
2,2021-06-15,8321.529271,17012.659354,12744.569866,1020253,NaN,Santiago
3,2021-06-16,9872.628696,18703.024312,14287.205309,1020253,NaN,Santiago
4,2021-06-17,10764.097397,19676.243419,15291.635632,1020253,NaN,Santiago


# Predicción Iterativa  

## Iteración 1 zona

In [38]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])



Listado_de_zonas=data['Zona de ventas.1'].unique

start_time = time()



#for Z in Listado_de_zonas:
Z='Norte'

dfz=Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz[dfz['Día natural'] >datetime.date(2021, 6, 2)]['Oficina de Ventas Unifica'].unique()

for y in Listado_de_Sucursales:
    #y='Huechuraba'
    dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


    Listado_de_Materiales=dfy['Material'].unique()

    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        material_time=time()
        print(y,m)


        dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

        dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
        dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
        dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df2=pd.concat([df2, dfmPred])
        
        print('El producto demoró',(time() - material_time),Z,y)
        print('tiempo que lleva corriendo',(time() - start_time)/60,'Min')
    
print('tiempo bloque completo',(time() - start_time)/60,'Min')
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Antofagasta 1020253


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.


El producto demoró 3.024878740310669 Norte Antofagasta
tiempo que lleva corriendo 0.05513916412989298 Min
Antofagasta 1020042


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.386347770690918 Norte Copiapo
tiempo que lleva corriendo 4.242494340737661 Min
Copiapo 1010048


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.904385805130005 Norte Copiapo
tiempo que lleva corriendo 4.324242417017619 Min
Copiapo 1010002
El producto demoró 4.662334680557251 Norte Copiapo
tiempo que lleva corriendo 4.401956315835317 Min
tiempo bloque completo 4.401964632670085 Min


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-13,-174.195975,334.712580,74.823879,1020253,Antofagasta,Norte
1,2021-06-14,-175.273428,360.578114,101.708623,1020253,Antofagasta,Norte
2,2021-06-15,-167.131355,361.639567,98.074634,1020253,Antofagasta,Norte
3,2021-06-16,-140.048748,386.602900,123.959807,1020253,Antofagasta,Norte
4,2021-06-17,-81.303926,451.291054,170.962713,1020253,Antofagasta,Norte


In [39]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [40]:
165.4798605442047/60

2.7579976757367453

### CSV SAVE 

In [41]:

min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2019-06-20
2021-06-18


In [42]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Zona '+Z+max_time+'.csv', index=False)

## Iteración completa Zona, Sucursal

In [43]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Sucursales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Oficina de Ventas Unifica'].unique()

    for y in Listado_de_Sucursales:
        #y='Huechuraba'
        print(y)
        dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


        Listado_de_Materiales=dfy[dfy.Año==2021].Material.unique()

        for m in Listado_de_Materiales:
            #m=Listado_de_Materiales[0]
            material_time=time()
            #print(y,m)


            dfm=Df_ubic(dfy,
                        'Material',m).groupby('Día natural'
                                                 ).sum().reset_index()[['Día natural',y_col]]

            dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower',
                                        'yhat_upper', 'yhat']].tail(7)
            
            dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
            dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
            dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
            dfmPred=dfmPred.reset_index()
            dfmPred.pop('index')
            df2=pd.concat([df2, dfmPred])
            print('El producto demoró',(time() - material_time),Z,y)
            print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')
    
print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Huechuraba


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.371374845504761 Santiago Huechuraba
tiempo que lleva corriendo 0.07800501982371012 Min


...

El producto demoró 3.6394832134246826 Venta Directa VD FoodService
tiempo que lleva corriendo 24.60259707768758 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.0328445434570312 Venta Directa VD FoodService
tiempo que lleva corriendo 24.653144486745198 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.88793420791626 Venta Directa VD FoodService
tiempo que lleva corriendo 24.734618377685546 Min
El producto demoró 2.9879391193389893 Venta Directa VD FoodService
tiempo que lleva corriendo 24.784425512949625 Min
tiempo bloque completo 24.784425512949625 Min


In [44]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [45]:
df2.head()

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-12,91.381925,1583.779519,856.754470,1020253,Huechuraba,Santiago
1,2021-06-13,-145.784982,1393.674744,582.661490,1020253,Huechuraba,Santiago
2,2021-06-14,-315.513713,1250.637066,502.017351,1020253,Huechuraba,Santiago
3,2021-06-15,-148.530060,1435.645389,658.550101,1020253,Huechuraba,Santiago
4,2021-06-16,-8.504280,1597.944441,788.136734,1020253,Huechuraba,Santiago


### CSV SAVE 

In [46]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-02-05
2021-06-18


In [47]:
df2.to_csv('Data '+min_time+' Predicción CL ' +max_time+ 'Zona, Sucursal.csv', index=False)


### Exploracion del error 

In [48]:
Z

'Venta Directa'

In [49]:
y

'VD FoodService'

In [50]:
m

'1010048'

In [51]:
dfy['Material'].unique()

array(['1020253', '1010267', '1010078', '1010048'], dtype=object)

In [52]:
dfy[dfy.Año==2021]['Material'].unique()

array(['1020253', '1010267', '1010078', '1010048'], dtype=object)

In [53]:
dfy[dfy.Año==2021].groupby('Material').count()

,Sector,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,,
1010048,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
1010078,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1010267,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
1020253,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [54]:
dfy.groupby('Material').count()

,Sector,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,,
1010048,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270
1010078,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
1010267,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
1020253,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36


In [55]:
dfy_21=dfy[dfy.Año==2021]

In [56]:
dfy_21[dfy_21.Material=='1010695']

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año


In [57]:
429.91*692.60

297755.666

## Iteración Zona

In [58]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)


    #Crear un listado de los materiales disponibles en la dat

    Listado_de_Materiales=dfz[dfz.Año==2021].Material.unique()     


    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        material_time=time()
        #print(y,m)


        dfm=Df_ubic(dfz,
                    'Material',m).groupby('Día natural'
                                             ).sum().reset_index()[['Día natural',y_col]]

        dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower',
                                    'yhat_upper', 'yhat']].tail(7)
        #Material
        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
        
        #Sucursal
        #dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]

        #Zona
        dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]

        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df2=pd.concat([df2, dfmPred])
        print('El producto demoró',(time() - material_time),Z)
        print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 5.352535009384155 Santiago
tiempo que lleva corriendo 0.09369253317515056 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.350914239883423 Santiago
tiempo que lleva corriendo 0.16621609131495158 Min


...

El producto demoró 3.4740254878997803 Venta Directa
tiempo que lleva corriendo 5.342289670308431 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.3142168521881104 Venta Directa
tiempo que lleva corriendo 5.3975432276725765 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.6597166061401367 Venta Directa
tiempo que lleva corriendo 5.45854682524999 Min
El producto demoró 2.887115478515625 Venta Directa
tiempo que lleva corriendo 5.506665416558584 Min
tiempo bloque completo 5.5066737333933515 Min


In [59]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

### CSV SAVE 

In [60]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-02
2021-06-18


In [61]:
df2.to_csv('Data'+min_time+' Predicción ' +max_time+ 'Zona.csv', index=False)


# Canal

## Iteración completa Zona, Sucursal

In [ ]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Sucursales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Oficina de Ventas Unifica'].unique()

    for y in Listado_de_Sucursales:
        #y='Huechuraba'
        print(y)
        dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)
        
        
        Listado_de_Canales=dfy[dfy['Día natural'] 
                                  >datetime.date(2021, 6, 2)][
            'Tipo de cliente'].unique()
        
        for c in Listado_de_Canales:
            dfc=Df_ubic(dfy,'Tipo de cliente',c)

            #Crear un listado de los materiales disponibles en la data


            Listado_de_Materiales=dfc[dfc.Año==2021].Material.unique()     


            for m in Listado_de_Materiales:
                #m=Listado_de_Materiales[0]
                material_time=time()
                #print(y,m)


                dfm=Df_ubic(dfc,
                            'Material',m).groupby('Día natural'
                                                     ).sum().reset_index()[['Día natural',y_col]]

                dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower',
                                            'yhat_upper', 'yhat']].tail(7)
                #Material
                dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
                #Canal de ventas
                dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]
                
                #Sucursal
                dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
                
                #Zona
                dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
                dfmPred=dfmPred.reset_index()
                dfmPred.pop('index')
                df2=pd.concat([df2, dfmPred])
                print('El producto demoró',(time() - material_time),Z,y)
                print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Huechuraba


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.554025411605835 Santiago Huechuraba
tiempo que lleva corriendo 0.08132384618123373 Min


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 5.1000025272369385 Santiago Huechuraba
tiempo que lleva corriendo 2.545164386431376 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 2.7998154163360596 Santiago Huechuraba
tiempo que lleva corriendo 2.5918411016464233 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
Z

In [ ]:
y

In [ ]:
c

In [ ]:
m

In [ ]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

### CSV SAVE 

In [ ]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

In [ ]:
df2.to_csv('Data'+min_time+' Predicción Por CANAL ' +max_time+ 'Zona, Sucursal.csv', index=False)


## Iteración Zona

In [ ]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Canales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Tipo de cliente'].unique()
    
    
    for c in Listado_de_Canales:
        dfc=Df_ubic(dfz,'Tipo de cliente',c)

        #Crear un listado de los materiales disponibles en la data
        Listado_de_Materiales=dfc[dfc.Año==2021].Material.unique()     


        for m in Listado_de_Materiales:
            #m=Listado_de_Materiales[0]
            material_time=time()
            #print(y,m)


            dfm=Df_ubic(dfc,
                        'Material',m).groupby('Día natural'
                                                 ).sum().reset_index()[['Día natural',y_col]]

            dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower',
                                        'yhat_upper', 'yhat']].tail(7)
            #Material
            dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
            #Canal de ventas
            dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]

            #Sucursal
            #dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
            
            #Zona
            dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
            
            dfmPred=dfmPred.reset_index()
            dfmPred.pop('index')
            df2=pd.concat([df2, dfmPred])
            print('El producto demoró',(time() - material_time),Z,y)
            print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


In [ ]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

### CSV SAVE 

In [ ]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

In [ ]:
df2.to_csv('Data'+min_time+' Predicción Por CANAL ' +max_time+ 'Zona.csv', index=False)


# DEmanda

In [ ]:
#Nombre del producto
Fecha='08_06_21'

# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name='Data 08_06_21 Predicción CL 2021-06-14'
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=",",dtype=types_dict)
Data.head(1)

In [ ]:
Data.groupby('Material').sum()